# QUERIES

## SQL

A continuación presentamos el modelo entidad relación:

![Modelo entidad relación](diagrama_er.png)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db" 
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

A. Ver grados con su rama, área y universidad

In [13]:
df_1 = pd.read_sql(
"""
SELECT  
    g.id AS id_grado, 
    g.nombre AS grado, 
    a.nombre AS area, 
    r.nombre AS rama, 
    u.nombre AS universidad 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
ORDER BY r.nombre, g.nombre;
""", engine)
df_1.head()

,id_grado,grado,area,rama,universidad
0,26,Animacion,Diseño,Artes y Humanidades,Universidad Europea de Madrid
1,26,Animacion,Diseño,Artes y Humanidades,Universidad Europea de Madrid
2,3,Antropologia Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...
3,3,Antropologia Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Complutense de Madrid
4,1,Antropologia Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...


In [14]:
df_1[df_1['grado'].str.lower() == "matematica computacional"]

,id_grado,grado,area,rama,universidad
581,203,Matematica Computacional,Ingeniería Matemática,Ingeniería y Arquitectura,Universidad Carlos III de Madrid


B. Consulta de titulaciones posibles según la nota de admisión

In [3]:
df_2 = pd.read_sql(
    """
SELECT  
    g.nombre AS grado, 
    u.nombre AS universidad, 
    nc.nota AS nota_corte, 
    nc.año, 
    nc.convocatoria 
FROM nota_corte nc 
JOIN grado_ofertado go ON nc.id_grado_ofertado = go.id 
JOIN grado g ON go.id_grado = g.id 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
WHERE nc.nota <= 11.0 

ORDER BY nc.nota DESC; 
    """, engine)
df_2.head()

,grado,universidad,nota_corte,año,convocatoria
0,Ingeniería en Tecnologías Industriales,Universidad Carlos III de Madrid,11.0,2025,ordinaria
1,Ingeniería en Tecnologías Industriales,Universidad a Distancia de Madrid UDIMA,11.0,2025,ordinaria
2,Ingeniería en Tecnologías Industriales,Universidad de Alcalá,11.0,2025,ordinaria
3,Ingeniería en Tecnologías Industriales,Universidad Politécnica de Madrid,11.0,2025,ordinaria
4,Ingeniería en Tecnologías Industriales,Universidad Pontificia Comillas ICAIICADE,11.0,2025,ordinaria


C. Consulta de universidades públicas por rama de conocimiento

In [4]:
df_3 = pd.read_sql(
    """
SELECT DISTINCT  
    u.nombre AS universidad, 
    r.nombre AS rama 

FROM universidad u 
JOIN facultad f ON u.id = f.id_universidad 
JOIN grado_ofertado go ON f.id = go.id_facultad 
JOIN grado g ON go.id_grado = g.id 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 

WHERE u.tipo = 'Pública' 
  AND r.nombre = 'Ingeniería y Arquitectura' 
ORDER BY u.nombre; 
    """, engine)
df_3.head()

,universidad,rama
0,Universidad Autónoma de Madrid,Ingeniería y Arquitectura
1,Universidad Carlos III de Madrid,Ingeniería y Arquitectura
2,Universidad Complutense de Madrid,Ingeniería y Arquitectura
3,Universidad de Alcalá,Ingeniería y Arquitectura
4,Universidad Nacional de Educación a Distancia ...,Ingeniería y Arquitectura


D. Comparativa de grados similares según nota media

In [13]:
df_4 = pd.read_sql(
    """
SELECT  
    a.nombre AS area, 
    g.nombre AS grado, 
    ROUND(AVG(nc.nota), 2) AS nota_media 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN nota_corte nc ON go.id = nc.id_grado_ofertado 
GROUP BY a.nombre, g.nombre 

ORDER BY a.nombre, nota_media DESC; 
    """, engine)
df_4.head()

,area,grado,nota_media
0,Antropología Social y Cultural,Antropología Social y Cultural,5.00
1,Arquitectura,Fundamentos de Arquitectura y Urbanismo,8.64
2,Artes,Conservación y Restauración del Patrimonio Cul...,6.71
3,Artes,Bellas Artes,6.32
4,Artes,Composición Musical,5.00


E. Consultar grados de una misma facultad

In [27]:
df_5 = pd.read_sql(
    """
    SELECT  
        g.id AS id_grado, 
        g.nombre AS grado,
        u.nombre AS universidad,
        f.nombre AS factultad
    FROM universidad u
    JOIN facultad f ON f.id_universidad = u.id 
    JOIN grado_ofertado go ON go.id_facultad = f.id
    JOIN grado g ON g.id = go.id_grado
    WHERE f.id= 9
    ORDER BY g.nombre;
    """,
    engine
)

df_5.head()


,id_grado,grado,universidad,factultad
0,108,Bachelor of Urban Management,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
1,109,Estudios Urbanos,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
2,33,Geografía e Historia,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
3,34,Historia,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
4,111,Información y Documentación,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...


F. Consultar facultades de una universidad

In [28]:
df_6 = pd.read_sql(
    """
    SELECT  
        u.nombre AS universidad,
        f.nombre AS factultad
    FROM universidad u
    JOIN facultad f ON f.id_universidad = u.id 
    WHERE u.id= 1
    ORDER BY f.nombre;
    """,
    engine
)

df_6.head()


,universidad,factultad
0,Universidad Nacional de Educación a Distancia ...,Escuela Técnica Superior de Ingenieros Industr...
1,Universidad Nacional de Educación a Distancia ...,Facultad Ciencias Económicas y Empresariales
2,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias
3,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias Económicas y Empresariales
4,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias Políticas y Sociología


## Elasticsearch

In [29]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': 'f9d659a396d8', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QjqgR16TSLem8J1oCtOMtQ', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [30]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for descripciones
embeddings_descripcion = model.encode(df["descripcion"].values.tolist())

# Generate embeddings (vector representations) for salidas
embeddings_salidas = model.encode(df["salidas"].values.tolist())

/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


NameError: name 'df' is not defined

In [ ]:
import json
from elasticsearch.helpers import bulk

index_name="informacion_grados"

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings_descripcion[i + idx].tolist()
            doc["salidas_vector"]=embeddings_salidas[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, index_name)

### Queries sintácticas

In [ ]:
import json

res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "industrial"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "informática"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

### Queries léxcias

In [ ]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "match": {
            "descripcion": "computación"
        }
    }
)


print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comunicación"}},
                {"match": {"salidas": "medios audiovisuales"}}
            ],
            "must_not": [
                {"match": {"descripcion": "ingeniería"}},
                {"match": {"descripcion": "matemáticas"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=1,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "gestión"}},
                {"match": {"salidas": "administración de empresas"}}
            ],
            "must_not": [
                {"match": {"descripcion": "finanzas"}},
                {"match": {"salidas": "contabilidad"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comportamiento humano"}},
                {"match": {"salidas": "social"}}
            ],
            "must_not": [
                {"term": {"nombre.keyword": "Psicología"}},
                {"term": {"nombre.keyword": "Biología Sanitaria"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")

In [ ]:
res = es.search(
    index="informacion_grados",
    size=2,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "arte digital"}},
                {"match": {"salidas": "diseño"}}
            ],
            "must_not": [
                {"match": {"descripcion": "docencia"}},
                {"match": {"descripcion": "ingeniería"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")


### Queries semánticas

In [ ]:
query_sentence = "análisis de datos"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":5,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

## Graph/Sparql

Primero presentamos nuestra ontología que utiliza los prefijos:
- sch: http://schema.org
- rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
- xsd: http://www.w3.org/2001/XMLSchema#
- dat: http://example.org/universidadesMadrid#
- owl: http://www.w3.org/2002/07/owl#

![Diagrama de la ontología](grafo.png)

In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON

# URL de tu endpoint SPARQL de GraphDB
endpoint_url = "http://graphdbdb:7200/repositories/Practica_GESTDB"

# Crear el objeto de conexión
sparql = SPARQLWrapper(endpoint_url)


A. Consultar todas las escuelas con sus universidades y datos de
contacto

In [16]:
query1 = """ 
PREFIX sch: <https://schema.org/>

SELECT ?schoolName ?universityName ?email ?phone ?address ?postalCode ?web
WHERE {
  ?school a sch:School ;
        sch:legalName ?schoolName ;
        sch:containedInPlace ?university .
        OPTIONAL { ?school sch:email ?email }
        OPTIONAL { ?school sch:telephone ?phone }
        OPTIONAL { ?school sch:address ?address }
        OPTIONAL { ?school sch:url ?web }

        ?university a sch:CollegeOrUniversity ;
}
ORDER BY ?universityName
LIMIT 10
"""
# Asignar la consulta
sparql.setQuery(query1)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

for r in results["results"]["bindings"]:
    school = r.get("schoolName", {}).get("value", "N/A")
    university = r.get("universityName", {}).get("value", "N/A")
    email = r.get("email", {}).get("value", "N/A")
    phone = r.get("phone", {}).get("value", "N/A")
    address = r.get("address", {}).get("value", "N/A")
    web = r.get("web", {}).get("value", "N/A")
    
    print(f"Facultad: {school} ; Universidad: {university} ; Email: {email} ; Telefono: {phone} ; Direccion: {address} ; Web: {web}")

Facultad: Facultad de Comunicacion y Artes Campus de MadridPrincesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 03 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Derecho y de Relaciones Internacionales Campus Madrid Princesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 01 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Ciencias Sociales Campus MadridPrincesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 03 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Ciencias Lenguas y Educacion Campus MadridPrincesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 03 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Lenguas y Educacion Campus MadridPrincesa ; Universidad: N/A ; Email: informa@

B. Obtener la nota de corte más reciente de cada grado

In [17]:
query2 = """ 
PREFIX dat: <http://example.org/universidadesMadrid#>
PREFIX sch: <https://schema.org/>

SELECT ?program ?programName (MAX(?year) AS ?latestYear) ?cutOff ?call ?group
WHERE {
  ?cutOffScore a dat:CutOffScore ;
               dat:educational_program ?courseInstance ;
               dat:value ?cutOff ;
               dat:year ?year ;
               dat:call ?call ;
               dat:group ?group .
               
  ?courseInstance sch:about ?program .
  ?program sch:name ?programName .
}
GROUP BY ?program ?programName ?cutOff ?call ?group
ORDER BY ?programName
LIMIT 10
"""
# Asignar la consulta
sparql.setQuery(query2)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

for r in results["results"]["bindings"]:
    program_name = r.get("programName", {}).get("value", "N/A")
    latest_year = r.get("latestYear", {}).get("value", "N/A")
    cut_off = r.get("cutOff", {}).get("value", "N/A")
    call = r.get("call", {}).get("value", "N/A")
    group = r.get("group", {}).get("value", "N/A")
    
    print(f"Grado: {program_name} ; Año: {latest_year} ; Nota de corte: {cut_off} ; Convocatoria: {call} ; Grupo: {group}")

Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 5.0 ; Convocatoria: ordinaria ; Grupo: 1
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 5.0 ; Convocatoria: ordinaria ; Grupo: 2
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 5.0 ; Convocatoria: ordinaria ; Grupo: 3
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 5.0 ; Convocatoria: ordinaria ; Grupo: 4
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 5.0 ; Convocatoria: ordinaria ; Grupo: 5
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 8.15 ; Convocatoria: ordinaria ; Grupo: 1
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 6.6 ; Convocatoria: ordinaria ; Grupo: 2
Grado: Antropologia Social y Cultural ; Año: 2025 ; Nota de corte: 5.0 ; Convocatoria: ordinaria ; Grupo: 5
Grado: Antropologia Social y Cultural ; Año: 2025 ; Nota de corte: 5.0 ; Convoc

C. Listar todas las universidades y las escuelas que contienen


In [18]:
query3 = """ 
PREFIX sch: <https://schema.org/>

SELECT ?universityName ?schoolName
WHERE {
  ?university a sch:CollegeOrUniversity ;
              sch:legalName ?universityName ;
              sch:containsPlace ?school .
  ?school sch:legalName ?schoolName .
}
ORDER BY ?universityName
LIMIT 10
"""
# Asignar la consulta
sparql.setQuery(query3)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

for r in results["results"]["bindings"]:
    university = r.get("universityName", {}).get("value", "N/A")
    school = r.get("schoolName", {}).get("value", "N/A")
    
    print(f"Universidad: {university} ; Facultad: {school}")

Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Ciencias de la Salud Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Estudios Sociales y Lenguas Aplicadas Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Educacion
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Estudios Sociales y Lenguas Aplicadas
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Farmacia Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Veterinaria Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Escuela Poltecnica Superior Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Escuela Politecnica Superior Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Musica y Artes Escenica

D. Nota de corte promedio por área

In [19]:
query4 = """ 
PREFIX dat: <http://example.org/universidadesMadrid#>
PREFIX sch: <https://schema.org/>

SELECT ?categoryName (AVG(?value) AS ?avgCutOff)
WHERE {
  ?cut a dat:CutOffScore ;
       dat:value ?value ;
       dat:educational_program ?courseInstance .
       
  ?courseInstance sch:about ?program .
  ?program sch:occupationalCategory ?category .
  ?category sch:name ?categoryName .
}
GROUP BY ?categoryName
ORDER BY DESC(?avgCutOff)
"""
# Asignar la consulta
sparql.setQuery(query4)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

for r in results["results"]["bindings"]:
    category = r.get("categoryName", {}).get("value", "N/A")
    avg_cutoff = r.get("avgCutOff", {}).get("value", "N/A")
    
    print(f"Rama: {category} ; Nota de corte media: {avg_cutoff}")

Rama: Ingeniería Agrícola / Alimentaria / Forestal ; Nota de corte media: 9.5199995
Rama: Nutrición Humana y Dietética ; Nota de corte media: 8.73
Rama: Farmacia ; Nota de corte media: 8.2775
Rama: Odontología ; Nota de corte media: 8.2775
Rama: Fisioterapia ; Nota de corte media: 8.148461
Rama: Enfermería ; Nota de corte media: 8.143334
Rama: Ingeniería Biomédica ; Nota de corte media: 7.974444
Rama: Psicología ; Nota de corte media: 7.5833335
Rama: Biología ; Nota de corte media: 7.4371424
Rama: Matemáticas ; Nota de corte media: 7.1628575
Rama: Ingeniería Informática ; Nota de corte media: 7.075
Rama: Geografía, Urbanismo y Ordenación del Territorio ; Nota de corte media: 6.9900002
Rama: Medicina ; Nota de corte media: 6.923333
Rama: Arquitectura ; Nota de corte media: 6.535217
Rama: Logopedia ; Nota de corte media: 6.51
Rama: Veterinaria ; Nota de corte media: 6.4
Rama: Ciencias Ambientales ; Nota de corte media: 6.35
Rama: Geomática ; Nota de corte media: 6.2253847
Rama: Física ; 

E. Ver todos los programas ofrecidos en una escuela concreta

In [ ]:
query5 = """ 
PREFIX sch: <https://schema.org/>

SELECT ?schoolName ?programName ?categoryName ?universityName
WHERE {
  ?courseInstance a sch:CourseInstance ;
                  sch:about ?program ;
                  sch:location ?school .
  ?school sch:legalName ?schoolName ;
          sch:containedInPlace ?university .
  ?university sch:legalName ?universityName .
  ?program sch:name ?programName ;
           sch:occupationalCategory ?category .
  ?category sch:name ?categoryName .

  FILTER(STR(?universityName) = "Universidad Politécnica de Madrid")
  FILTER(CONTAINS(LCASE(STR(?schoolName)), "campus sur"))
}
ORDER BY ?programName
"""
# Asignar la consulta
sparql.setQuery(query5)
sparql.setReturnFormat(JSON)

# Ejecutar la consulta
results = sparql.query().convert()

for r in results["results"]["bindings"]:
    school = r.get("schoolName", {}).get("value", "N/A")
    university = r.get("universityName", {}).get("value", "N/A")
    program = r.get("programName", {}).get("value", "N/A")
    category = r.get("categoryName", {}).get("value", "N/A")
    
    print(f"Facultad: {school} ; Universidad: {university} ; Grado: {program} ; Área: {category}")

Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas Informaticos Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ciencia de Datos e Inteligencia Artificial ; Área: Ingeniería Informática
Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas de Telecomunicacion Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria Electronica de Comunicaciones ; Área: Ingeniería en Telecomunicación
Facultad: Escuela Tecnica Superior de Ingenieros en Topografia, Geodesia y Cartografia Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria Geomatica ; Área: Geomática
Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas Informaticos Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria Informatica ; Área: Ingeniería Informática
Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas de Telecomunicacion Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria 